In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Mathematical/data cleaning tools
import numpy as np
import pandas as pd
from collections import Counter
from pathlib import Path

#sklearn
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix

In [3]:
# Fast AI
import fastai
from fastai import *
from fastai.text import *
from fastai.text.data import *

### Reading in Data to Pandas

In [55]:
twitter = pd.read_csv('../data/full-corpus.csv')

In [56]:
# twitter = twitter.drop(columns=['Topic', 'TweetId', 'TweetDate'], axis=1)
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


### Reading in Data to FastAI

In [57]:
path = Path('/home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data')
path

PosixPath('/home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data')

In [68]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'full-corpus.csv', text_cols='TweetText', label_cols='Sentiment')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'full-corpus.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [69]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.130485,4.069437,0.288579,02:31


## Sentence Completion

In [70]:
learn.predict("This is a review about", n_words=10)

"This is a review about du Prix de France 's landscape , the"

In [71]:
learn.predict("I love my new iphone, the features are", n_words=10)

'I love my new iphone, the features are still time to give my love it , and how'

#### Save Encoder

In [72]:
learn.save_encoder('ft_enc')

## Sentiment Classification

In [73]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (4090 items)
x: TextList
xxbos xxunk,xxbos positive,xxbos xxunk,xxbos xxunk,xxbos xxunk
y: CategoryList
twitter,microsoft,google,microsoft,twitter
Path: /home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data;

Valid: LabelList (1023 items)
x: TextList
xxbos xxunk,xxbos xxunk,xxbos xxunk,xxbos xxunk,xxbos xxunk
y: CategoryList
microsoft,microsoft,google,apple,google
Path: /home/jarid/Galvanize_coursework/capstones/Twitter_Analysis/data;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(4552, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(4552, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM

In [74]:
data_clas.show_batch()

text,target
xxbos xxunk,twitter
xxbos xxunk,apple
xxbos xxunk,apple
xxbos xxunk,twitter
xxbos xxunk,twitter


In [75]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.372572,1.343656,0.311828,00:29


In [78]:
learn.predict("I love my Xbox!")

(Category apple, tensor(0), tensor([0.3348, 0.2388, 0.2856, 0.1408]))